In [ ]:
%matplotlib tk
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1,'..')
import tools
import os
import SIKE2
import numpy as np
import matplotlib.pyplot as plt
import sk_plotting_functions as spf
from post_processing import *

In [ ]:
skrun = spf.SKRun('/Users/dpower/Documents/01 - PhD/14 - ELM investigation/01 - Runs/01 - Equilibria/02 - Kinetic/P_in = 64MW/Output_job_EQ_K64_2e19')
skrun.load_dist()
f0 = skrun.data['DIST_F'][0] * skrun.n_norm / skrun.v_th ** 3
Te = skrun.data['TEMPERATURE']  * skrun.T_norm
ne = skrun.data['DENSITY']  * skrun.n_norm
skvgrid = skrun.vgrid 
skxgrid = skrun.xgrid

In [ ]:
el = 'Li'

# max_x = -1
# r = SIKE.SIKERun(xgrid=skxgrid[:max_x], fe=f0[:,:max_x], vgrid=skvgrid, opts={'modelled_impurities': ['He'], 
#                                                             'EVOLVE': True, 
#                                                             'ionization': True,
#                                                             'radiative recombination': False,
#                                                             'excitation': True,
#                                                             'emission': False,
#                                                             'autoionization': False,
#                                                             'delta_t': 100.0,
#                                                             'max_steps': 1, 
#                                                             'state_ids': None})
num_x = 200
r = SIKE2.SIKERun(xgrid=np.linspace(0,10,num_x), Te=np.geomspace(1.0,100,num_x), ne=1e18 * np.ones(num_x), opts={'modelled_impurities': [el], 
                                                            'EVOLVE': False, 
                                                            'ionization': True,
                                                            'radiative recombination': True,
                                                            'excitation': True,
                                                            'emission': True,
                                                            'autoionization': False,
                                                            'delta_t': 100.0,
                                                            'max_steps': 1, 
                                                            'state_ids': None})

In [ ]:
r.run()

In [ ]:
# Plot Z_avg
n = r.impurities[el].dens_Max
Zavg = get_Zavg(n,r.impurities[el].states, r.num_x)
fig,ax = plt.subplots(1)
ax.plot(r.Te*r.T_norm,Zavg)
ax.grid()

In [ ]:
# Plot all states on x grid
n = r.impurities[el].dens_Max
fig,ax = plt.subplots(1)
for i in range(len(n[0,:])):
    ax.plot(r.xgrid,n[:,i])
# ax.set_yscale('log')

In [ ]:
# Plot state distribution
n = r.impurities[el].dens_Max
x_pos = 0
n_gnorm = np.zeros(r.impurities[el].tot_states)
lev_E = np.zeros(r.impurities[el].tot_states)
for i in range(r.impurities[el].tot_states):
    n_gnorm[i] = n[x_pos,i] / r.impurities[el].states[i].stat_weight
    lev_E[i] = r.impurities[el].states[i].energy
fig,ax = plt.subplots(1)
ax.plot(lev_E,n_gnorm)
ax.set_yscale('log')

# Compare rad-rec rates

In [ ]:
# Compare radiative recombination rates
import aurora
import re
import scipy.interpolate

r = SIKE.SIKERun(xgrid=np.linspace(0,10,100), Te=np.linspace(1.0,10,100), ne=1e20 * np.ones(100), opts={'modelled_impurities': ['H'], 
                                                            'EVOLVE': True, 
                                                            'ionization': True,
                                                            'radiative recombination': True,
                                                            'excitation': True,
                                                            'emission': True,
                                                            'autoionization': True,
                                                            'delta_t': 100000.0,
                                                            'max_steps': 1000, 
                                                            'state_ids': [0,100]})
r.run()

rr_trans = r.impurities['H'].transitions[1]
fac_radrec_rates = np.zeros(r.num_x)
for i in range(r.num_x):
    fac_radrec_rates[i] = rr_trans.get_mat_value(r.fe_Max[:,i],r.vgrid,r.dvc) / r.ne[i]

In [ ]:
H_radrec_file = '/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/imp_data/Hydrogen/nrb05##_h1ls.dat'

def lmom(l):
    if l == '0':
        return 'S'
    elif l == '1':
        return 'P'
    elif l == '2':
        return 'D'
    elif l == '3':
        return 'F'
    elif l == '4':
        return 'G'
    elif l == '5':
        return 'H'
    elif l == '6':
        return 'I'
    elif l == '7':
        return 'J'
    elif l == '8':
        return 'K'
    elif l == '9':
        return 'L'
    elif l == '10':
        return 'M'
    elif l == '11':
        return 'N'
    elif l == '12':
        return 'O'

def get_adas_statename(line):
    shell_re = re.findall('([123456789][SPDFGHIJKLMNOP][123456789])', line)
    shell = (','.join(shell_re)).lower()
    mom = re.search('\(\d\)\d\(', line)
    s = mom[0][1]
    l = lmom(mom[0][3])
    if shell == '':
        return s + l
    else:
        return shell + ' ' + s + l

def load_adas_radrec_rates(adas_file, from_state, to_state, T_norm, n_norm, t_norm):

    with open(adas_file) as f:
        lines = f.readlines()

    # Get parent and child states
    started_to = False
    for i, l in enumerate(lines):
        if 'PARENT TERM INDEXING' in l:
            from_start = i+4
        if 'LS RESOLVED TERM INDEXING' in l:
            from_end = i-1
            started_to = True
            to_start = i+4
        if started_to:
            if l == ' \n' or l == '\n':
                to_end = i
                break
    from_states = []
    to_states = []
    for l in lines[from_start:from_end]:
        from_states.append(get_adas_statename(l))
    for l in lines[to_start:to_end]:
        to_states.append(get_adas_statename(l))

    # Find the number of lines over which index / data points are written
    for i, l in enumerate(lines):
        if 'INDX TE=' in l and 'PRTI=' in lines[i-2]:
            idx_line = i
            break
    for i, l in enumerate(lines[idx_line:]):
        if l == '\n' or l == ' \n':
            idx_end = i + idx_line
            break
    block = ''.join(lines[idx_line:idx_end])
    ch_idx_re = re.findall('( \d+ )', block)
    num_ch = len(ch_idx_re)
    lines_per_entry = int((idx_end - idx_line - 2) / num_ch)

    for i, l in enumerate(lines):
        if 'INDX TE=' in l and 'PRTI=' in lines[i-2]:
            # TODO: What happens if the index is over 3 lines? Need to see an example for format
            T_idx_lines = ' '.join(lines[i:i+lines_per_entry])
            T_idx_matches = re.findall('(\d.\d+[ED][+-]\d+)', T_idx_lines)
            Te = np.array([float(T.replace('D', 'E')) for T in T_idx_matches])
            break

    K2eV = 11603.247217
    Te = Te / (K2eV * T_norm)  # Convert to eV and normalise

    # Get transitions
    started_trans = False
    from_trans = []
    to_trans = []
    for i, l in enumerate(lines):
        if i > 4:
            if 'INDX TE=' in l and 'PRTI=' in lines[i-2]:
                started_trans = True
                from_trans.append(i+2)
            if started_trans:
                if l == ' \n' or l == '\n':
                    to_trans.append(i)
                    started_trans = False
    searching = True
    for i, from_idx in enumerate(from_trans):
        to_idx = to_trans[i]
        parent_idx = i
        if searching:
            for j, l in enumerate(lines[from_idx:to_idx:lines_per_entry]):
                child_idx = int(l.split()[0]) - 1
                if from_states[parent_idx] == from_state and to_states[child_idx] == to_state:
                    rate_data = (' '.join(lines[from_idx + j *
                                                lines_per_entry:from_idx + (j+1)*lines_per_entry])).split()
                    rates = np.array([float(r.replace('D', 'E'))
                                     for r in rate_data[1:]])
                    searching = False
                    break
    rates = rates * t_norm * n_norm * 1e-6  # Normalise

    return rates, Te

radrec_rate, radrec_Te = load_adas_radrec_rates(H_radrec_file, '1S', '1s1 2S', r.T_norm, r.n_norm, r.t_norm)
radrec_interp = scipy.interpolate.interp1d(radrec_Te, radrec_rate, fill_value='extrapolate')

adas_radrec_rates = np.zeros(r.num_x)
for i,Te in enumerate(r.Te):
    adas_radrec_rates[i] = max(radrec_interp(Te), 0.0)

In [ ]:
fig,ax = plt.subplots(1)
ax.plot(r.Te*r.T_norm, fac_radrec_rates)
ax.plot(r.Te*r.T_norm, adas_radrec_rates)

In [ ]:
el = 'He'
cooling_curves = get_cooling_curves(r, el, kinetic=False)

In [ ]:
fig,ax = plt.subplots(1)
for z in range(2):
    ax.plot(r.Te*r.T_norm,cooling_curves[:,z],label=z)
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid()

In [ ]:
from petsc4py import PETSc
import petsc4py
petsc4py.init()

In [ ]:
rate_mat = PETSc.Mat().createAIJ([1000, 1000], nnz=100)
rate_mat.setValue(0,0,1.0,addv=True)

In [ ]:
rate_mat.setValue(0,0,1.0,addv=True)

In [ ]:
rate_mat.assemblyBegin()
rate_mat.assemblyEnd()

In [ ]:
rate_mat.getValue(0,0)

In [ ]:
ksp = PETSc.KSP().create()

In [ ]:
ksp.getConvergedReason()